In [ ]:
"----------------Python script to fetch all the tweets of the user from his/her timeline------------------------------------- "
"----------------------------------------Author Subodh Rai-------------------------------------------------------------- "
"----------------------------------Mail subodhrai898@gmail.com------------------------------------------------------------"
"Use Tweepy module for twitter API"
"Use Jsonline module to dump all tweets in .jsonl file"
"Function used here get_tweets() for getting list "
"Use Pandas Dataframes to show and store data in tabular form"
"Please Read Comment to understand the code"

In [25]:
#This block contain all imports and API Secret/Key 
#Required module for this project
import tweepy,json                                               #tweepy use for twitter API and Json to handle json data
import pandas as pd                                              #Pandas use for Dataframe and show information in tabular form
import jsonlines                                                 #jsonlines module use to store json in .jsonl format that contain one line per json information
from datetime import datetime                                    #this use to parse data
access_token = "2164637652-uGDxiZ8aPaThVexak2nkX8FLnoDEv2i4UOuj5yo"                 #Access Token
access_secret = "vRGBewKNdGRkorFwIUjJJIpFfvXBBfy2EnI2OEZRfCwRR"                     #Access Token Secret
consumer_key = "MrITPRcLi6Wp4YMnB0ygjpI4r"                                          #Consumer Key used
consumer_secret = "5dDMlcoLuj4eYYuZ2vNE1Dqs1fyqKKEbDpfr410fFkKU9THKUO"              #Consumer Key Secret

#This all use in OAuth in twitter API

In [15]:
#This function store list of tweets(data) in txt file here data is string of json fromat and file is name 

def store_txt(data,file):
    with open(file,'w+') as f:
        f.write(data)
    

In [16]:
#This function return list of tweets store in file in txt and return string tweet_str

def load_txt(file):
    tweets_str=""
    with open(file,'r') as f:
        tweets_str = f.read()
    return tweets_str

In [ ]:
"get_tweets is our main function which call Rest Api of twitter and api.user_timeline used to fetch all the tweets"
"Here I used loop which extract tweet in bunch of 200 tweets at one time untill in next iteration we will not get any tweets"
"I used extended mode to get all information of the tweet including images details and full text"
"Please read the comment to understand more"

In [17]:
#This is our main function that fetch all tweets of the user specify by username 


def get_tweets(username):
    # Authorization to consumer key and consumer secret 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
        # Access to user's access key and access secret 
        auth.set_access_token(access_token, access_secret) 
  
        # Calling api 
        api = tweepy.API(auth)
        #List that store all tweets of the user
        alltweets = []
        #use api.user_timeline to fetch all tweets timeline
        #we can also user Cursor for it but this somewhat controlable
        #api.user_timeline take three argument and we use tweet_mode='extended' so that we get all information of image and media
        #here we fetch first 200 tweets
        new_tweets = api.user_timeline(screen_name=username, count=200,tweet_mode='extended')
        #Extend our list
        alltweets.extend(new_tweets)
        #get ID of our last tweets so it use in max_ID
        oldest = alltweets[-1].id - 1
        #this loop will help to extract all tweets from the user
        while len(new_tweets) > 0:
                    #Here we use same function api.user_timeline but use max_id so that next time we fetch tweets after that ID
                    new_tweets = api.user_timeline(screen_name=username, count=200, max_id=oldest,tweet_mode='extended')
                    alltweets.extend(new_tweets)
                    oldest = alltweets[-1].id - 1
        #return list of tweets
        return alltweets

In [18]:
#This function will help to dump our alltweets list into jsonlines file 

def store_jsonl(alltweets,file):
    writer = jsonlines.open(file, mode='w') #open jsonline file
    for tweet in alltweets:
            writer.write(tweet._json)      #dump one by one json of each tweet to file
    writer.close()                         #Always close the stream

In [26]:
#This function I used to help me to visualize the content of alltweets in readable indented fashion store in a txt file.
def store_in_txt(alltweets,file):       #alltweets is list of tweets
        data1 = dict()                  #use dict to create only one json file where each tweet is fetched by number - 'tweet_no'
        tweet_no = 1
        
       
        for tweet in alltweets:                #Store All tweets in that dictionary with key to each tweet is tweet_no
            data1[tweet_no] = tweet._json      #convert tweet to json
            tweet_no+=1                        #update tweet_number
       
        json_str = json.dumps(data1,indent=4)  #dump it in string json_str with indentation equals to 4
        store_txt(json_str,file)          
       

In [ ]:
"Here I used Pandas Dataframe to show and store data in Tabular from"
"Dataframes have columns Text , Like , Retweet_Count ,Number of Image"
"Here I used Datetime as index and create a Time Series"
"The benefit of Time Series Data is that we can easily analyse it"
"Please read the comment to understand more"


In [32]:
#This function return dataframe that contain Time-series Data of Text , Like, Retweet_count and Number of Image


def Create_DataFrame(tweets):        #take iterator of tweets here tweet is jsonline iterator
    
        
        df = pd.DataFrame()          #create empty dataframe
       
        for result in tweets:        #iterate through each json line
                                      
            total_image = 0          #Use to count number of Image
            try:
                   total_image = len(result['entities']['media'])            #Image and video store in entitied->Media
            except KeyError:                                             
                   total_image = None                                        #Use try except to catch None image
            
            try:
                    tota_image = len(result['extended_entities']['media'])   #This is important as earlier if we use simple mode we get only some image list
                                                                             #The extend_entities->media will hold list of all images and work only in extended mode
                    if(total_image==None):
                        total_image=tota_image                               #We add total of image
                    else:          
                        total_image+=tota_image
                
            except KeyError:
                   pass
                
            if(total_image!=None):                                            #We decrease count by one as it will include th profile picture also
                total_image-=1
                
            
            dm = {"Text":[result["full_text"]],                               #Our one row of Data frame      
              "Date":[pd.to_datetime(result["created_at"][4:], format='%b %d %H:%M:%S +0000 %Y', utc=True)
                      ], #This pd.to_dateTime will parse our date time so that we can use as index
              "Like":[result["favorite_count"]],                               #Number of Likes 
              "Retweet_count":[result["retweet_count"]],                       #Number of Retweet
            "Number_image":[total_image]}                                      #Total Images
            #Append Row to dataframe df                                                 
            df = df.append(pd.DataFrame(dm,columns=['Text', 'Date', 'Like','Retweet_count','Number_image']))
        #Reset index and index data to Date and create a Time Series Data
        df.set_index(["Date"], inplace = True) 
        return df

In [33]:
#This function use to convert jsonl to dataframe
def jsonl_to_dataframe(file):
    reader = jsonlines.open(file,'r')
    df = Create_DataFrame(reader)               #Call Create_DataFrame
    reader.close()
    return df                                   #Return Dataframe

In [34]:
#Time to Compile my whole Work
filejson = "data.jsonl"                                 #File name of our .jsonl
filetxt = "data.txt"                                    #File name of our .txt
username="midasIIITD"                                   #User name of the screen/handle
tweets = get_tweets(username)                           #Start to get all tweets list
store_jsonl(tweets,filejson)                            #dumo in jsonline file
store_in_txt(tweets,filetxt)


In [35]:
df = jsonl_to_dataframe(filejson)                       #Convert Jsonline to DataFrame

In [30]:
df.info()                                               #Information about our DataFram

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 333 entries, 2019-04-07 06:55:19+00:00 to 2018-07-23 12:53:15+00:00
Data columns (total 4 columns):
Text             333 non-null object
Like             333 non-null int64
Retweet_count    333 non-null int64
No_image         47 non-null object
dtypes: int64(2), object(2)
memory usage: 13.0+ KB


In [41]:
df.head(333)

,Text,Like,Retweet_count,Number_image
Date,,,,
2019-04-07 06:55:19+00:00,"Other queries: ""none of the Tweeter Apis give ...",3,2,None
2019-04-07 06:53:38+00:00,"Other queries: ""do we have to make two differe...",3,1,None
2019-04-07 05:32:27+00:00,"Other queries: ""If using Twitter api, it does ...",4,1,None
2019-04-07 05:29:40+00:00,Response to some queries asked by students on ...,6,1,None
2019-04-06 17:11:29+00:00,RT @kdnuggets: Top 8 #Free Must-Read #Books on...,0,2,None
2019-04-06 16:43:27+00:00,@nupur_baghel @PennDATS Congratulation @nupur_...,14,3,1
2019-04-05 16:08:37+00:00,We have emailed the task details to all candid...,10,1,None
2019-04-05 04:05:11+00:00,RT @rfpvjr: Our NAACL paper on polarization in...,0,16,None
2019-04-05 04:04:43+00:00,RT @kdnuggets: Effective Transfer Learning For...,0,10,1


In [40]:
df.to_csv("Table.csv")